# Fine-tuning leve com BERT — Classificação de Sentimento (IMDb reduzido)

Notebook didático em Português. Objetivo: demonstrar o fluxo de fine-tuning de um modelo pré-treinado (BERT-base) em um subset pequeno do IMDb, com poucas épocas para aula.

In [ ]:
!pip install -q transformers datasets evaluate accelerate torch scikit-learn


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1) Imports e configuração

In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print('Device:', device)

c:\Users\rlotte\Documents\projects\bootcamp\semana-2\dia-5\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cpu


## 2) Carregar dataset (IMDb reduzido)

In [13]:
raw = load_dataset("imdb")
train_small = raw['train'].select(range(1000))
test_small = raw['test'].select(range(200))
dataset = {"train": train_small, "test": test_small}
print(dataset['train'][0])

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

## 3) Tokenização

In [15]:
MODEL_NAME = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

tokenized_train = dataset['train'].map(tokenize_fn, batched=True)
tokenized_test = dataset['test'].map(tokenize_fn, batched=True)

tokenized_train = tokenized_train.remove_columns([c for c in tokenized_train.column_names if c not in ['input_ids','attention_mask','label']])
tokenized_test = tokenized_test.remove_columns([c for c in tokenized_test.column_names if c not in ['input_ids','attention_mask','label']])

tokenized_train.set_format(type="torch", columns=["input_ids","attention_mask","label"])
tokenized_test.set_format(type="torch", columns=["input_ids","attention_mask","label"])

print(tokenized_train[0])

{'label': tensor(0), 'input_ids': tensor([  101,  1045, 12524,  1045,  2572,  8025,  1011,  3756,  2013,  2026,
         2678,  3573,  2138,  1997,  2035,  1996,  6704,  2008,  5129,  2009,
         2043,  2009,  2001,  2034,  2207,  1999,  3476,  1012,  1045,  2036,
         2657,  2008,  2012,  2034,  2009,  2001,  8243,  2011,  1057,  1012,
         1055,  1012,  8205,  2065,  2009,  2412,  2699,  2000,  4607,  2023,
         2406,  1010,  3568,  2108,  1037,  5470,  1997,  3152,  2641,  1000,
         6801,  1000,  1045,  2428,  2018,  2000,  2156,  2023,  2005,  2870,
         1012,  1026,  7987,  1013,  1028,  1026,  7987,  1013,  1028,  1996,
         5436,  2003,  8857,  2105,  1037,  2402,  4467,  3689,  3076,  2315,
        14229,  2040,  4122,  2000,  4553,  2673,  2016,  2064,  2055,  2166,
         1012,  1999,  3327,  2016,  4122,  2000,  3579,  2014,  3086,  2015,
         2000,  2437,  2070,  4066,  1997,  4516,  2006,  2054,  1996,  2779,
        25430, 14728,  2245,  

## 4) Carregar modelo pré-treinado

In [5]:
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(device)
print(model.config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "dtype": "float32",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.57.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



## 5) Setup de treino com Trainer

In [16]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
            "f1": f1.compute(predictions=preds, references=labels, average='weighted')["f1"]}

In [17]:
training_args = TrainingArguments(
    output_dir="./bert_sentiment",
    eval_strategy="epoch",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./bert_logs",
    logging_steps=50,
    save_strategy="no",
    load_best_model_at_end=False,
    report_to="none"
)

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

C:\Users\rlotte\AppData\Local\Temp\ipykernel_10076\3635770616.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## 6) Treinamento (rápido)

In [19]:
trainer.train()

c:\Users\rlotte\Documents\projects\bootcamp\semana-2\dia-5\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

## 7) Avaliação e testes rápidos

In [ ]:
preds = trainer.predict(tokenized_test)
print('Métricas de teste:', preds.metrics)

In [ ]:
def predict_text(text):
    enc = tokenizer(text, truncation=True, padding='max_length', max_length=128, return_tensors='pt').to(device)
    model.eval()
    with torch.no_grad():
        out = model(**enc)
        pred = torch.argmax(out.logits, dim=-1).item()
    return pred

ex = "This movie was really entertaining and I loved it."
print('Exemplo:', ex)
print('Predição (1=pos, 0=neg):', predict_text(ex))

## 8) Sugestões de experimento
- Aumentar `num_train_epochs` para 3–4
- Testar `bert-base-multilingual-cased`
- Congelar embeddings: `model.bert.embeddings.requires_grad_(False)` e treinar só o head
- Integrar W&B: `report_to='wandb'` no `TrainingArguments`